In [13]:
import pandas as pd
import numpy as np
from random import sample

df = pd.read_csv('podcasts_entries.csv', header=None,error_bad_lines=False)
    
rt_cnt=len(set(df[1])) 
us_cnt=len(set(df[3]))
users= dict()
movies= dict()
id_to_pd=dict()
id_to_us=dict()
print us_cnt
    
utility=np.zeros((us_cnt,rt_cnt))
us_cnt_tr=0
mv_cnt_tr=0
for i in range(0, len(df)):
    if df[3][i] not in users:
        users[df[3][i]]= us_cnt_tr
        id_to_us[us_cnt_tr]= df[3][i]
        us_cnt_tr=us_cnt_tr+1
    if df[1][i] not in movies:
        movies[df[1][i]]= mv_cnt_tr
        id_to_pd[mv_cnt_tr]=df[1][i]
        mv_cnt_tr=mv_cnt_tr+1
    idx_us= users[df[3][i]]
    idx_mv= movies[df[1][i]]
    utility[idx_us][idx_mv]=1.0
    #print utility[0]
s=np.sum(utility, axis=1, keepdims=True)
count=0
for i in range(0, len(s)):
    if s[i] > 1.0:
        count=count+1

us_cnt_tr=0
user_to_id= dict()
id_to_user=dict()
test_data=[]
utility_nw=np.zeros((count,rt_cnt))
for i in range(0, len(utility)):
    if s[i] > 1:
        user_to_id[id_to_us[i]]=us_cnt_tr
        id_to_user[us_cnt_tr]= id_to_us[i]
        non_zero= np.nonzero(utility[i])
        non_zero=non_zero[0]
        if s[i] > 10:
            #print len(non_zero[2])
            length= len(non_zero)
            test_len= (length/10)*3
            train_len= length- test_len
            test_ind = sample(range(length),test_len)
            #print length
            for j in range(0, len(non_zero)):
                if j not in test_ind:
                    utility_nw[us_cnt_tr][non_zero[j]]= utility[i][non_zero[j]]
                else:
                    ls=[us_cnt_tr, non_zero[j]]
                    test_data.append(ls)
        else:
            for j in non_zero:
                utility_nw[us_cnt_tr][j]= utility[i][j]
        us_cnt_tr=us_cnt_tr+1    
        
print len(utility_nw)
fp= open("users.txt","w")
fp.writelines(str(users))
fp.close()
    
fp= open("ratings.txt","w")
fp.writelines(str(movies))
fp.close()
    
np.savetxt('test.txt', s, delimiter=',')
np.savetxt('utl_nw.txt', utility_nw, delimiter=',')

Skipping line 74: expected 4 fields, saw 5
Skipping line 300: expected 4 fields, saw 6
Skipping line 634: expected 4 fields, saw 5
Skipping line 766: expected 4 fields, saw 5
Skipping line 895: expected 4 fields, saw 5
Skipping line 1377: expected 4 fields, saw 5
Skipping line 1558: expected 4 fields, saw 5
Skipping line 1752: expected 4 fields, saw 5
Skipping line 1959: expected 4 fields, saw 5
Skipping line 2031: expected 4 fields, saw 5
Skipping line 2099: expected 4 fields, saw 5
Skipping line 2156: expected 4 fields, saw 5
Skipping line 2195: expected 4 fields, saw 5
Skipping line 2257: expected 4 fields, saw 5
Skipping line 2259: expected 4 fields, saw 5
Skipping line 2293: expected 4 fields, saw 5
Skipping line 2394: expected 4 fields, saw 5
Skipping line 2407: expected 4 fields, saw 5
Skipping line 2540: expected 4 fields, saw 5
Skipping line 2545: expected 4 fields, saw 5
Skipping line 2942: expected 4 fields, saw 5
Skipping line 2977: expected 4 fields, saw 5
Skipping line 31

8344
992


In [15]:
print len(test_data)

225


In [16]:
from sklearn.metrics import jaccard_similarity_score

sim_mat= np.zeros((count,count))
for i in range(0,count):
    for j in range(0, count):
        sim_mat[i][j]= jaccard_similarity_score(utility_nw[i], utility_nw[j])

In [19]:
idx_mv=5
ind = np.argpartition(sim_mat[idx_mv], -50)[-50:]

best_rated=np.zeros((rt_cnt))
for i in ind:
    for j in range(0, rt_cnt):
        if i != idx_mv:
            best_rated[j]=  best_rated[j]+ utility_nw[i][j]
ind_best = np.argpartition(best_rated, -10)[-10:]
print ind_best

for i in ind_best:
    print utility_nw[idx_mv][i]

[640 126 335 130 477 818 475 697  63 621]
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0


In [32]:
from sklearn.metrics import jaccard_similarity_score

sim_mat= np.zeros((count,count))
for i in range(0,count):
    for j in range(0, count):
        sim_mat[i][j]= jaccard_similarity_score(utility_nw[i], utility_nw[j])

In [57]:
predicted_rtngs=[]
for i in range(0,len(test_data)):
    idx_us=test_data[i][0]
    idx_mv=test_data[i][1]
    vj= utility_nw[:,[idx_mv]]
    vj=np.transpose(vj)
    vj_all=np.dot(vj,sim_mat[idx_us])
    rtng= round(0.1*vj_all[0])
    if rtng > 1:
        rtng=1
    predicted_rtngs.append(rtng)
    print 'userId '+str(test_data[i][0])+' movieId '+str(test_data[i][1])+' actual rating 1 rating predicted '+str(rtng)


userId 3 movieId 0 actual rating 1 rating predicted 0.0
userId 3 movieId 3 actual rating 1 rating predicted 0.0
userId 3 movieId 6 actual rating 1 rating predicted 0.0
userId 3 movieId 10 actual rating 1 rating predicted 0.0
userId 3 movieId 16 actual rating 1 rating predicted 0.0
userId 3 movieId 167 actual rating 1 rating predicted 1.0
userId 4 movieId 2 actual rating 1 rating predicted 0.0
userId 4 movieId 4 actual rating 1 rating predicted 1.0
userId 4 movieId 29 actual rating 1 rating predicted 1.0
userId 12 movieId 103 actual rating 1 rating predicted 0.0
userId 12 movieId 155 actual rating 1 rating predicted 1.0
userId 12 movieId 206 actual rating 1 rating predicted 0.0
userId 16 movieId 63 actual rating 1 rating predicted 1
userId 16 movieId 130 actual rating 1 rating predicted 1.0
userId 16 movieId 143 actual rating 1 rating predicted 0.0
userId 16 movieId 206 actual rating 1 rating predicted 0.0
userId 16 movieId 358 actual rating 1 rating predicted 0.0
userId 16 movieId 683 

In [59]:
import math
rmse=0
for i in range(0,len(predicted_rtngs)):
    rmse=rmse+pow((1-predicted_rtngs[i]), 2)
rmse=rmse/len(predicted_rtngs)
rmse=math.sqrt(rmse)
print 'Root Mean Squared Error is '+str(rmse)

Root Mean Squared Error is 0.85114302232
